In [86]:
import requests
from bs4 import BeautifulSoup
import json
import re
import string
import sys
from bs4.element import NavigableString, Tag
from os import path

In [87]:
basepath = path.abspath('')
item_filepath = path.abspath(path.join(basepath, "..", "game_data", "roll_20_all_items.json"))
f = open(item_filepath, "r", encoding="utf-8")
all_items = json.load(f)
f.close()

spell_filepath = path.abspath(path.join(basepath, "..", "game_data", "roll_20_all_spells.json"))
fs = open(spell_filepath, "r", encoding="utf-8")
all_spells = json.load(fs)
fs.close()
    
legal_traits = ["jack of all trades", "pact of the blade", "step of the wind", "mask of the wild", "pact of the tome",
               "fury of the small", "book of ancient secrets"]
banned_traits = ["you have"]

class DualSpecError(Exception):
    pass

class NoTraitsError(Exception):
    pass

class NoInvError(Exception):
    pass

class NoProfError(Exception):
    pass

class BadItemError(Exception):
    pass

modifiers = [-5, -5, -4, -4, -3, -3, -2, -2, -1, -1, 0, 0, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 6, 6, 7, 7, 8, 8, 9, 9, 10]
attr_keys = {"STR": "strength", "DEX": "dexterity", "CON": "constitution", 
             "INT": "intelligence", "WIS": "wisdom", "CHA": "charisma"}
spellcasting_mod = {"bard": "CHA", "paladin": "CHA", "sorcerer": "CHA", "warlock": "CHA",
                   "cleric": "WIS", "ranger": "WIS", "wizard": "INT", "eldritch knight": "INT"}

In [110]:
url_list = ["http://www.kassoon.com/dnd/5e/character-sheet/846/elliza-maren-drow-elf-warlock-1/",
           "http://www.kassoon.com/dnd/5e/character-sheet/2384/vinheim-high-elf-wizard-1/",
           "http://www.kassoon.com/dnd/5e/character-sheet/2382/mijira-imbixtellrhyst-dragonborn-rogue-1/",
           "http://www.kassoon.com/dnd/5e/character-sheet/208/adam-fidge-variant-human-bard-5/",
           "https://www.kassoon.com//dnd/5e/character-sheet/48/paren-galastacia-wood-elf-rogue-1/",
           "http://www.kassoon.com/dnd/5e/character-sheet/55/dodge-lightfoot-halfling-sorcerer2/",
           "https://www.kassoon.com/dnd/5e/character-sheet/28/eston-human-bard-1-cleric-1/",
           "https://www.kassoon.com/dnd/5e/character-sheet/2100/",
           "https://www.kassoon.com/dnd/5e/character-sheet/95/"]

In [120]:
# Scrape a single page
def scrape_page(url):
    char = {}
    page = requests.get(url)

    # Basic Info #
    soup = BeautifulSoup(page.content, "html.parser")
    char["name"] = soup.find("div", {"class": "character_name"}).contents[0]
    char["link"] = url
    char_box = soup.findAll("div", {"class": "underline autocomplete updstats"})
    cls = char_box[0].contents[0]
    if "/" in cls:
        raise DualSpecError
    match = re.match(r"([A-Za-z]+)\s?([0-9]+)", cls, re.I)
    if match:
        items = match.groups()
        char["class"] = items[0].lower()
        char["level"] = int(items[1])
    else:
        char["class"] = cls.rsplit(" ")[0].lower()
        char["level"] = int(cls.rsplit(" ")[1])
    char["background"] = char_box[1].contents[0].lower()
    char["race"] = char_box[2].contents[0].lower()
    char["alignment"] = char_box[3].contents[0].lower()
    char["gender"] = char_box[4].contents[0].lower()

    # Attributes #
    attrs = soup.findAll("div", {"class": "character_attribute_box"})
    char["attributes"] = {}
    for att in attrs[:6]:
        att_map = {}
        att_name = att.find("div", {"class": "character_attr_name"}).contents[0]
        att_map["true_val"] = int(att.find("div", {"class": "character_attr_value"}).contents[0])
        att_map["points"] = int(att.find("div", {"class": "character_attr_value editshow updstats"}).contents[0])
        att_map["racial_bonus"] = att_map["true_val"] - att_map["points"]
        att_map["modifier"] = modifiers[att_map["true_val"]]
        char["attributes"][att_name.lower()] = att_map
    char["attributes"]["armor_class"] = 10 + char["attributes"]["dexterity"]["modifier"]
    char["attributes"]["initiative"] = char["attributes"]["dexterity"]["modifier"]
    
    sheet_proficiency = int(attrs[8].find("div", {"class": "character_attr_bonus"}).contents[0])
    char["attributes"]["proficiency"] = int((char["level"]-1)/4 + 2)
    proficiency_diff = char["attributes"]["proficiency"] - sheet_proficiency
    
    char["attributes"]["perception"] = 10 + char["attributes"]["wisdom"]["modifier"]
    char["attributes"]["hit_points"] = int(attrs[11].find("div", {"id": "val15"}).contents[0])
    
    # Skills #
    skill_map = {}
    proficient_skills = []
    skills = soup.findAll("div", {"class": "clear flex"})
    for s in skills[1:-1]:
        skill_name = ((s.find("div", {"class": "character_skills_info"}).contents[0]).split("(")[0]).replace("* ", "").replace("'", "").strip().lower()
        skill_val = 0
        try:
            skill_val = int(s.find("span", {"class": "editshow character_skills_bonus underline"}).contents[0])
        except AttributeError:
            print("ERROR: " + skill_name)
            print(s)
            skill_val = 0
        if s.find("input", {"checked": "checked"}):
            proficient_skills.append(string.capwords(skill_name))
            skill_val = skill_val + proficiency_diff
        skill_map[string.capwords(skill_name)] = skill_val
    
    char["skills"] = skill_map
    char["proficient_skills"] = proficient_skills
    
    # Saves #
    saves = soup.find("div", {"class": "character_throws"}).findAll("div", {"class": "clear"})
    saving_throws = {}
    proficient_saves = []
    for s in saves[:-1]:
        save_name = s.find("div", {"class": "character_throw_info"}).contents[0].lower()
        save_val = char["attributes"][save_name]["modifier"]
        if s.find("input", {"checked": "checked"}):
            proficient_saves.append(save_name)
            save_val = save_val + char["attributes"]["proficiency"]
        saving_throws[save_name] = save_val
    char["saving_throws"] = saving_throws
    char["proficient_saves"] = proficient_saves
    
    
    # Attacks #
    misc = soup.find("div", {"class": "character_misc"})
    atks_and_spells = []
    item_list = misc.find("tbody", {"id": "list0"}).findAll("tr")
    for item in item_list:
        item_map = {}
        i = item.findAll("td")
        #print(i)
        #print(type(i[0].contents[0]))
        
        if (isinstance(i[0].contents[0], Tag)):
            if str(i[0].contents[0].contents[0]) == "Roll":
                continue
        item_map["name"] = string.capwords(i[0].contents[0])
        item_name = item_map["name"].title()
        
        # Item and spell mapping
        def map_item_or_spell(item_map, item_name):
            if item_name in all_items:
                if "Damage" in all_items[item_name]:
                    item_map["damage"] = all_items[item_name]["Damage"]
                if "Damage Type" in all_items[item_name]:
                    item_map["damage_type"] = all_items[item_name]["Damage Type"]
                else:
                    item_map["damage_type"] = "normal"
                if "Range" in all_items[item_name]:
                    item_map["range"] = all_items[item_name]["Range"]
                else:
                    item_map["range"] = "Reach"
                if "Properties" in all_items[item_name]:
                    item_map["properties"] = all_items[item_name]["Properties"]
                return (item_map, True)
            elif item_name in all_spells:
                if "Damage" in all_spells[item_name]:
                    item_map["damage"] = all_spells[item_name]["Damage"]
                if "Damage Type" in all_spells[item_name]:
                    item_map["damage_type"] = all_spells[item_name]["Damage Type"]
                if "Range" in all_spells[item_name]:    
                    item_map["range"] = all_spells[item_name]["Range"]
                else:
                    item_map["range"] = "Reach"
                return (item_map, True)
            else:
                return (item_map, False)
        
        item_map, v = map_item_or_spell(item_map, item_name)
        if not v:
            new_name = string.capwords(item_name.replace(" ", "").title())
            item_map, v2 = map_item_or_spell(item_map, new_name)
            if not v2:
                raise BadItemError("Bad Item or Spell: " + str(item_name))
            else:
                item_map["name"] = new_name.title()
        
        #print(item_map)
        atks_and_spells.append(item_map)
    
    char["attacks"] = atks_and_spells
    
    
    # Spellcasting #
    if char["class"] in spellcasting_mod:
        spellcasting = {}
        spellcasting["spellcast_modifier"] = attr_keys[spellcasting_mod[char["class"]]]
        spellcasting["spellsave_dc"] = 8 + char["attributes"][spellcasting["spellcast_modifier"]]["modifier"] + char["attributes"]["proficiency"]
        spellcasting["spell_attack_bonus"] = int(char["attributes"][spellcasting["spellcast_modifier"]]["modifier"] + char["attributes"]["proficiency"])
        for i in range(1,10):
            slot_val = misc.find("div", {"id": ("val" + str(55+i))}).contents
            if slot_val:
                if slot_val[0].find("-") != -1:
                    slot_val[0] = int(slot_val[0].split('-')[0])
                    

                spellcasting["level_" + str(i) + "_slots"] = int(slot_val[0])
            else:
                spellcasting["level_" + str(i) + "_slots"] = 0
        char["spellcasting"] = spellcasting
        
        
    # Features & Traits #
    features = soup.findAll("div", {"class": "edithide"})
    #print(feats[3].contents)
    features_and_traits = []
    pattern = re.compile("^([A-Z][a-z]+):")
    for i in features[3].contents:
        i = str(i).strip().replace("’", "'")
        #print(i)
        if ":" in i:
            #print(i)
            #print("")
            pattern = re.search("^([A-Za-z' ]+):", i)
            bold_pattern = re.search("\<b\>([A-Za-z':, ]+)\</b\>", i)
            p_pattern = re.search("^\<p\>([A-Za-z', ]+):", i)
            pat_str = ""
            if pattern:
                pat_str = string.capwords(str(pattern.group(0)).rstrip(":"))
            elif bold_pattern:
                pat_str = string.capwords(str(bold_pattern.group(0)).title().split("<B>")[1].rsplit(":</B>")[0])
            elif p_pattern:
                pat_str = string.capwords(str(p_pattern.group(0)).title().split("<P>")[1].rsplit(":")[0])
            
            if (pat_str != "") and (pat_str.lower() not in banned_traits):
                if pat_str not in features_and_traits:
                    features_and_traits.append(pat_str)
        
        else:
            spl = i.split()
            i = " ".join(spl)
            basic_pattern = re.search("^([A-Za-z', ]+)$", i)
            if basic_pattern:
                if i.lower() in legal_traits:
                    pat_str = string.capwords(str(basic_pattern.group(0)))
                    if pat_str not in features_and_traits:
                        features_and_traits.append(pat_str)
                elif "," in i.lower():
                    if ("draconic ancestry" in i.lower() or "favored enemy" in i.lower()):
                        pat_str = string.capwords(str(basic_pattern.group(0)))
                        if pat_str not in features_and_traits:
                            features_and_traits.append(pat_str)
                    else:
                        # print("Rejected: " + " ".join(spl) + " link: " + url)
                        pass
                elif len(spl) > 3:
                    # print("Rejected: " + " ".join(spl) + " link: " + url)
                    pass
                else:
                    pat_str = string.capwords(str(basic_pattern.group(0)))
                    if pat_str not in features_and_traits:
                        features_and_traits.append(pat_str)
            
            #else:
                #print("No Pattern: " + " ".join(spl))
                
    if not features_and_traits:
        raise NoTraitsError
    char["features"] = (features_and_traits)
    
    
    # Inventory #
    inv_list = []
    inv = soup.findAll("div", {"class": "character_panel"})[3].find("div", {"id": "val46"})
    # print(inv.contents)
    for item in inv:
        item_dict = {}
        if str(item) != "<br/>":
            # print(item)
            item_ct = re.search(r'\d+', item)
            if item_ct:
                item_ct = item_ct.group()
                item_name = re.split(r'\d+', item)[1].strip()
                if item_name[-1:] == "s":
                    item_name = item_name[:-1]
                item_dict["name"] = item_name
                item_dict["count"] = int(item_ct)
            else:
                item_dict["name"] = item
                item_dict["count"] = 1
            inv_list.append(item_dict)
    if len(inv_list) <= 0:
        raise NoInvError
    char["inventory"] = inv_list
    
    gold = soup.findAll("div", {"class": "character_panel"})[4].find("div", {"id": "val45"})
    # print(gold.contents)
    if len(gold.contents) > 0:
        gold_cont = re.search(r'\d+', gold.contents[0])
        if gold_cont:
            char["gp"] = int(gold_cont.group())
        else:
            raise BadItemError("Bad gp value: '" + str(gold.contents[0]) + "'")
    else:
        char["gp"] = 0
        
        
    # Proficiencies & Languages #
    profs = soup.findAll("div", {"class": "character_panel"})[5].find("div", {"id": "val47"})
    # print(profs.contents)
    prof_list = []
    lang_list = []
    for p in profs:
        if str(p) != "<br/>":
            p = string.capwords(p)
            if "Proficiencies" in p:
                pr = p.replace("Proficiencies", "", 1).strip(": ").split(",")
                for i in pr:
                    prof_list.append(i.strip())
                    
            if "Languages" in p:
                pr = p.replace("Languages", "", 1).strip(": ").split(",")
                for i in pr:
                    lang_list.append(i.strip())
                    
    if not prof_list or not lang_list:
        raise NoProfError
    char["proficiencies"] = prof_list
    char["languages"] = lang_list
                
    
    # End of char #
    return char

In [122]:
# TESTING #
print(json.dumps(scrape_page(url_list[7]), indent=4))

{
    "name": "Greethen Norixius",
    "link": "https://www.kassoon.com/dnd/5e/character-sheet/2100/",
    "class": "rogue",
    "level": 1,
    "background": "inheritor",
    "race": "human",
    "alignment": "neutral",
    "gender": "male",
    "attributes": {
        "strength": {
            "true_val": 13,
            "points": 12,
            "racial_bonus": 1,
            "modifier": 1
        },
        "dexterity": {
            "true_val": 16,
            "points": 15,
            "racial_bonus": 1,
            "modifier": 3
        },
        "constitution": {
            "true_val": 14,
            "points": 13,
            "racial_bonus": 1,
            "modifier": 2
        },
        "intelligence": {
            "true_val": 11,
            "points": 10,
            "racial_bonus": 1,
            "modifier": 0
        },
        "wisdom": {
            "true_val": 15,
            "points": 14,
            "racial_bonus": 1,
            "modifier": 2
        },
        "c

In [118]:
test_characters = []
for idx, url in enumerate(url_list):
    try:
        test_characters.append(scrape_page(url))
    except ValueError:
        print("ValError at idx " + str(idx) + " link: " + url)
    except IndexError:
        print("IndexError at idx " + str(idx) + " link: " + url)
    except DualSpecError:
        print("DualSpecError at idx " + str(idx) + " link: " + url)
    except NoTraitsError:
        print("NoTraitsError at idx " + str(idx) + " link: " + url)
    except NoInvError:
        print("NoInvError at idx " + str(idx) + " link: " + url)
    except NoProfError:
        print("NoProfError at idx " + str(idx) + " link: " + url)
    except BadItemError as e:
        print("BadItemError at idx " + str(idx) + " link: " + url)
print(json.dumps(test_characters, indent=4))
with open("scraper_output/kassoon_scrapes/test_kassoon_out.json", "w", encoding='utf-8') as outfile:
    json.dump(test_characters, outfile, ensure_ascii=False, indent=4)

NoProfError at idx 3 link: http://www.kassoon.com/dnd/5e/character-sheet/208/adam-fidge-variant-human-bard-5/
NoTraitsError at idx 5 link: http://www.kassoon.com/dnd/5e/character-sheet/55/dodge-lightfoot-halfling-sorcerer2/
DualSpecError at idx 6 link: https://www.kassoon.com/dnd/5e/character-sheet/28/eston-human-bard-1-cleric-1/
BadItemError at idx 8 link: https://www.kassoon.com/dnd/5e/character-sheet/95/
[
    {
        "name": "Elliza Maren",
        "class": "warlock",
        "level": 1,
        "background": "mercenary veteran",
        "race": "drow elf",
        "alignment": "neutral",
        "gender": "female",
        "attributes": {
            "strength": {
                "true_val": 8,
                "points": 8,
                "racial_bonus": 0,
                "modifier": -1
            },
            "dexterity": {
                "true_val": 16,
                "points": 14,
                "racial_bonus": 2,
                "modifier": 3
            },
          

In [127]:
# Scrape by sheet number
num_base = "https://www.kassoon.com/dnd/5e/character-sheet/"
all_chars = []
out_count = 99
total_valid = 0
total_attempts = 0
start = 0
to_check = 15000
cap = 10000
class_count = {}
level_count = {}
for i in range(start, start + to_check):
    total_attempts = total_attempts + 1
    try:
        #print("i = " + str(i))
        all_chars.append(scrape_page(num_base + str(i) + "/"))
        total_valid = total_valid + 1
        if all_chars[-1]["class"] not in class_count:
            class_count[all_chars[-1]["class"]] = 1
        else:
            class_count[all_chars[-1]["class"]] += 1

        if all_chars[-1]["level"] not in level_count:
            level_count[all_chars[-1]["level"]] = 1
        else:
            level_count[all_chars[-1]["level"]] += 1

    except ValueError:
        print("ValError at idx " + str(i) + " link: " + num_base + str(i) + "/")
    except IndexError:
        print("IndexError at idx " + str(i) + " link: " + num_base + str(i) + "/")
    except DualSpecError:
        print("DualSpecError at idx " + str(i) + " link: " + num_base + str(i) + "/")
    except NoTraitsError:
        print("NoTraitsError at idx " + str(i) + " link: " + num_base + str(i) + "/")
    except NoInvError:
        print("NoInvError at idx " + str(i) + " link: " + num_base + str(i) + "/")
    except NoProfError:
        print("NoProfError at idx " + str(i) + " link: " + num_base + str(i) + "/")
    except BadItemError as e:
        print(e)
        print("BadItemError at idx " + str(i) + " link: " + num_base + str(i) + "/")
    
    if len(all_chars) >= 100:
        outf = "scraper_output/kassoon_scrapes/kassoon_out_" + str(out_count) + ".json"
        with open(outf, "w", encoding='utf-8') as outfile:
            json.dump(all_chars, outfile, ensure_ascii=False, indent=4)
        print("")
        print("Printing page " + str(out_count) + " to file")
        print("Total Attempts: " + str(total_attempts))
        print("Total Valid: " + str(total_valid))
        print("")
        print("Class Count:")
        print(class_count)
        print("Level Count:")
        print(level_count)
        print("")
        out_count = out_count + 1
        all_chars = []
    if total_valid >= cap:
        break
if(len(all_chars) > 0):
    print("")
    print("Printing page " + str(out_count) + " to file")

    outf = "scraper_output/kassoon_scrapes/kassoon_out_" + str(out_count) + ".json"
    with open(outf, "w", encoding='utf-8') as outfile:
        json.dump(all_chars, outfile, ensure_ascii=False, indent=4)        

print("Final Count!")
print("Total Attempts: " + str(total_attempts))
print("Total Valid: " + str(total_valid))
print("")
print("Class Count:")
print(class_count)
print("Level Count:")
print(level_count)

DualSpecError at idx 28 link: https://www.kassoon.com/dnd/5e/character-sheet/28/
NoInvError at idx 30 link: https://www.kassoon.com/dnd/5e/character-sheet/30/
NoTraitsError at idx 36 link: https://www.kassoon.com/dnd/5e/character-sheet/36/
Bad Item or Spell: Shield Bash
BadItemError at idx 40 link: https://www.kassoon.com/dnd/5e/character-sheet/40/
Bad Item or Spell: Unarmed Strike
BadItemError at idx 46 link: https://www.kassoon.com/dnd/5e/character-sheet/46/
Bad Item or Spell: Staff Of Defense
BadItemError at idx 54 link: https://www.kassoon.com/dnd/5e/character-sheet/54/
NoTraitsError at idx 55 link: https://www.kassoon.com/dnd/5e/character-sheet/55/
NoTraitsError at idx 56 link: https://www.kassoon.com/dnd/5e/character-sheet/56/
Bad Item or Spell: Dagger (Thrown)
BadItemError at idx 58 link: https://www.kassoon.com/dnd/5e/character-sheet/58/
Bad Item or Spell: 2 Dagger
BadItemError at idx 61 link: https://www.kassoon.com/dnd/5e/character-sheet/61/
Bad Item or Spell: Sneak Attack
Ba

Bad Item or Spell: Spell Attack
BadItemError at idx 226 link: https://www.kassoon.com/dnd/5e/character-sheet/226/
Bad Item or Spell: Spell Attack
BadItemError at idx 227 link: https://www.kassoon.com/dnd/5e/character-sheet/227/
Bad Item or Spell: Fist
BadItemError at idx 229 link: https://www.kassoon.com/dnd/5e/character-sheet/229/
Bad Item or Spell: Spell Attack
BadItemError at idx 230 link: https://www.kassoon.com/dnd/5e/character-sheet/230/
Bad Item or Spell: Unarmed
BadItemError at idx 232 link: https://www.kassoon.com/dnd/5e/character-sheet/232/
Bad Item or Spell: Spell Attack
BadItemError at idx 233 link: https://www.kassoon.com/dnd/5e/character-sheet/233/
Bad Item or Spell: Spell Attack
BadItemError at idx 236 link: https://www.kassoon.com/dnd/5e/character-sheet/236/
Bad Item or Spell: Crossbow
BadItemError at idx 238 link: https://www.kassoon.com/dnd/5e/character-sheet/238/
NoTraitsError at idx 239 link: https://www.kassoon.com/dnd/5e/character-sheet/239/
Bad Item or Spell: Cro

NoInvError at idx 351 link: https://www.kassoon.com/dnd/5e/character-sheet/351/
Bad Item or Spell: Spell Attack
BadItemError at idx 355 link: https://www.kassoon.com/dnd/5e/character-sheet/355/
IndexError at idx 356 link: https://www.kassoon.com/dnd/5e/character-sheet/356/
Bad Item or Spell: Spell Attack
BadItemError at idx 357 link: https://www.kassoon.com/dnd/5e/character-sheet/357/
Bad Item or Spell: Spell Attack
BadItemError at idx 358 link: https://www.kassoon.com/dnd/5e/character-sheet/358/
Bad Item or Spell: Spell Attack
BadItemError at idx 362 link: https://www.kassoon.com/dnd/5e/character-sheet/362/
Bad Item or Spell: Witch Bolt
BadItemError at idx 364 link: https://www.kassoon.com/dnd/5e/character-sheet/364/
Bad Item or Spell: Spell Attack
BadItemError at idx 365 link: https://www.kassoon.com/dnd/5e/character-sheet/365/
Bad Item or Spell: Spell Attack
BadItemError at idx 367 link: https://www.kassoon.com/dnd/5e/character-sheet/367/
Bad Item or Spell: Spell Attack
BadItemError

Bad Item or Spell: Spell Attack
BadItemError at idx 477 link: https://www.kassoon.com/dnd/5e/character-sheet/477/
Bad Item or Spell: %0%
BadItemError at idx 480 link: https://www.kassoon.com/dnd/5e/character-sheet/480/
Bad Item or Spell: Spell Attack
BadItemError at idx 481 link: https://www.kassoon.com/dnd/5e/character-sheet/481/
NoProfError at idx 482 link: https://www.kassoon.com/dnd/5e/character-sheet/482/
Bad Item or Spell: Spell Attack
BadItemError at idx 483 link: https://www.kassoon.com/dnd/5e/character-sheet/483/
IndexError at idx 484 link: https://www.kassoon.com/dnd/5e/character-sheet/484/
IndexError at idx 485 link: https://www.kassoon.com/dnd/5e/character-sheet/485/
Bad Item or Spell: Spell Attack
BadItemError at idx 486 link: https://www.kassoon.com/dnd/5e/character-sheet/486/
IndexError at idx 487 link: https://www.kassoon.com/dnd/5e/character-sheet/487/
Bad Item or Spell: Spell Attack
BadItemError at idx 489 link: https://www.kassoon.com/dnd/5e/character-sheet/489/
Bad 

ValError at idx 597 link: https://www.kassoon.com/dnd/5e/character-sheet/597/
Bad Item or Spell: Spell Attack
BadItemError at idx 598 link: https://www.kassoon.com/dnd/5e/character-sheet/598/
Bad Item or Spell: Spell Attack
BadItemError at idx 600 link: https://www.kassoon.com/dnd/5e/character-sheet/600/
Bad Item or Spell: Spell Attack
BadItemError at idx 602 link: https://www.kassoon.com/dnd/5e/character-sheet/602/
IndexError at idx 603 link: https://www.kassoon.com/dnd/5e/character-sheet/603/
Bad Item or Spell: Spell Attack
BadItemError at idx 604 link: https://www.kassoon.com/dnd/5e/character-sheet/604/
NoTraitsError at idx 605 link: https://www.kassoon.com/dnd/5e/character-sheet/605/
Bad Item or Spell: Spell Attack
BadItemError at idx 606 link: https://www.kassoon.com/dnd/5e/character-sheet/606/
Bad Item or Spell: Spell Attack
BadItemError at idx 608 link: https://www.kassoon.com/dnd/5e/character-sheet/608/
Bad Item or Spell: Spell Attack
BadItemError at idx 609 link: https://www.k

...


Printing page 26 to file
Total Attempts: 4281
Total Valid: 2700

Class Count:
{'druid': 214, 'paladin': 237, 'sorcerer': 174, 'cleric': 214, 'wizard': 226, 'ranger': 233, 'barbarian': 257, 'monk': 211, 'fighter': 274, 'rogue': 274, 'warlock': 187, 'bard': 198, 'apothecary': 1}
Level Count:
{1: 2511, 3: 49, 4: 23, 2: 23, 7: 6, 5: 38, 20: 10, 10: 12, 9: 4, 16: 1, 15: 3, 12: 1, 6: 10, 8: 6, 14: 1, 18: 1, 17: 1}

NoTraitsError at idx 4304 link: https://www.kassoon.com/dnd/5e/character-sheet/4304/

Printing page 27 to file
Total Attempts: 4382
Total Valid: 2800

Class Count:
{'druid': 224, 'paladin': 243, 'sorcerer': 181, 'cleric': 221, 'wizard': 236, 'ranger': 244, 'barbarian': 266, 'monk': 219, 'fighter': 281, 'rogue': 281, 'warlock': 190, 'bard': 213, 'apothecary': 1}
Level Count:
{1: 2611, 3: 49, 4: 23, 2: 23, 7: 6, 5: 38, 20: 10, 10: 12, 9: 4, 16: 1, 15: 3, 12: 1, 6: 10, 8: 6, 14: 1, 18: 1, 17: 1}

NoTraitsError at idx 4388 link: https://www.kassoon.com/dnd/5e/character-sheet/4388/
No

NoTraitsError at idx 5986 link: https://www.kassoon.com/dnd/5e/character-sheet/5986/

Printing page 43 to file
Total Attempts: 5996
Total Valid: 4400

Class Count:
{'druid': 355, 'paladin': 377, 'sorcerer': 306, 'cleric': 361, 'wizard': 392, 'ranger': 379, 'barbarian': 393, 'monk': 359, 'fighter': 417, 'rogue': 407, 'warlock': 324, 'bard': 329, 'apothecary': 1}
Level Count:
{1: 4211, 3: 49, 4: 23, 2: 23, 7: 6, 5: 38, 20: 10, 10: 12, 9: 4, 16: 1, 15: 3, 12: 1, 6: 10, 8: 6, 14: 1, 18: 1, 17: 1}

NoTraitsError at idx 6072 link: https://www.kassoon.com/dnd/5e/character-sheet/6072/
NoTraitsError at idx 6091 link: https://www.kassoon.com/dnd/5e/character-sheet/6091/

Printing page 44 to file
Total Attempts: 6098
Total Valid: 4500

Class Count:
{'druid': 364, 'paladin': 380, 'sorcerer': 316, 'cleric': 368, 'wizard': 398, 'ranger': 385, 'barbarian': 400, 'monk': 367, 'fighter': 424, 'rogue': 421, 'warlock': 334, 'bard': 342, 'apothecary': 1}
Level Count:
{1: 4311, 3: 49, 4: 23, 2: 23, 7: 6, 5:

NoTraitsError at idx 7631 link: https://www.kassoon.com/dnd/5e/character-sheet/7631/
NoTraitsError at idx 7659 link: https://www.kassoon.com/dnd/5e/character-sheet/7659/
NoTraitsError at idx 7708 link: https://www.kassoon.com/dnd/5e/character-sheet/7708/

Printing page 60 to file
Total Attempts: 7712
Total Valid: 6100

Class Count:
{'druid': 500, 'paladin': 525, 'sorcerer': 448, 'cleric': 498, 'wizard': 539, 'ranger': 511, 'barbarian': 545, 'monk': 520, 'fighter': 551, 'rogue': 544, 'warlock': 443, 'bard': 475, 'apothecary': 1}
Level Count:
{1: 5911, 3: 49, 4: 23, 2: 23, 7: 6, 5: 38, 20: 10, 10: 12, 9: 4, 16: 1, 15: 3, 12: 1, 6: 10, 8: 6, 14: 1, 18: 1, 17: 1}


Printing page 61 to file
Total Attempts: 7812
Total Valid: 6200

Class Count:
{'druid': 508, 'paladin': 533, 'sorcerer': 461, 'cleric': 507, 'wizard': 546, 'ranger': 521, 'barbarian': 549, 'monk': 532, 'fighter': 558, 'rogue': 555, 'warlock': 447, 'bard': 482, 'apothecary': 1}
Level Count:
{1: 6011, 3: 49, 4: 23, 2: 23, 7: 6, 5:


Printing page 78 to file
Total Attempts: 9518
Total Valid: 7900

Class Count:
{'druid': 654, 'paladin': 682, 'sorcerer': 585, 'cleric': 659, 'wizard': 687, 'ranger': 661, 'barbarian': 705, 'monk': 672, 'fighter': 701, 'rogue': 681, 'warlock': 581, 'bard': 631, 'apothecary': 1}
Level Count:
{1: 7711, 3: 49, 4: 23, 2: 23, 7: 6, 5: 38, 20: 10, 10: 12, 9: 4, 16: 1, 15: 3, 12: 1, 6: 10, 8: 6, 14: 1, 18: 1, 17: 1}

NoTraitsError at idx 9552 link: https://www.kassoon.com/dnd/5e/character-sheet/9552/
NoTraitsError at idx 9553 link: https://www.kassoon.com/dnd/5e/character-sheet/9553/

Printing page 79 to file
Total Attempts: 9620
Total Valid: 8000

Class Count:
{'druid': 663, 'paladin': 692, 'sorcerer': 591, 'cleric': 666, 'wizard': 696, 'ranger': 671, 'barbarian': 712, 'monk': 678, 'fighter': 709, 'rogue': 689, 'warlock': 588, 'bard': 644, 'apothecary': 1}
Level Count:
{1: 7811, 3: 49, 4: 23, 2: 23, 7: 6, 5: 38, 20: 10, 10: 12, 9: 4, 16: 1, 15: 3, 12: 1, 6: 10, 8: 6, 14: 1, 18: 1, 17: 1}


P

NoTraitsError at idx 11292 link: https://www.kassoon.com/dnd/5e/character-sheet/11292/
NoTraitsError at idx 11314 link: https://www.kassoon.com/dnd/5e/character-sheet/11314/

Printing page 96 to file
Total Attempts: 11332
Total Valid: 9700

Class Count:
{'druid': 797, 'paladin': 846, 'sorcerer': 738, 'cleric': 798, 'wizard': 852, 'ranger': 808, 'barbarian': 848, 'monk': 813, 'fighter': 871, 'rogue': 838, 'warlock': 720, 'bard': 770, 'apothecary': 1}
Level Count:
{1: 9511, 3: 49, 4: 23, 2: 23, 7: 6, 5: 38, 20: 10, 10: 12, 9: 4, 16: 1, 15: 3, 12: 1, 6: 10, 8: 6, 14: 1, 18: 1, 17: 1}

NoTraitsError at idx 11357 link: https://www.kassoon.com/dnd/5e/character-sheet/11357/

Printing page 97 to file
Total Attempts: 11433
Total Valid: 9800

Class Count:
{'druid': 809, 'paladin': 856, 'sorcerer': 747, 'cleric': 805, 'wizard': 862, 'ranger': 819, 'barbarian': 854, 'monk': 819, 'fighter': 879, 'rogue': 845, 'warlock': 727, 'bard': 777, 'apothecary': 1}
Level Count:
{1: 9611, 3: 49, 4: 23, 2: 23, 

In [ ]:
# Scrape by page
master_page = requests.get("https://www.kassoon.com/dnd/5e/premade-characters/")
link_soup = BeautifulSoup(master_page.content, "html.parser")
chr_list = link_soup.find("tbody", {"id": "tblChrList"}).findAll("tr")
#print(chr_list[0])
base = "http://www.kassoon.com"

cap_page = 1000
all_chars_page = []
for idx,c in enumerate(chr_list):
    c_url = c.find("a", href=True)["href"]
    try:
        all_chars_page.append(scrape_page(base + c_url))
    except ValueError:
        print("ValError at idx " + str(idx) + " link: " + base + c_url)
    except IndexError:
        print("IndexError at idx " + str(idx) + " link: " + base + c_url)
    except DualSpecError:
        print("DualSpecError at idx " + str(idx) + " link: " + base + c_url)
    except NoTraitsError:
        print("NoTraitsError at idx " + str(idx) + " link: " + base + c_url)
    except NoInvError:
        print("NoInvError at idx " + str(idx) + " link: " + base + c_url)
    except NoProfError:
        print("NoProfError at idx " + str(idx) + " link: " + base + c_url)
    except BadItemError as e:
        print(e)
        print("BadItemError at idx " + str(idx) + " link: " + base + c_url)
    if len(all_chars_page) >= cap_page:
        break


In [100]:
# premade page test
original_stdout = sys.stdout
with open("temp_out.txt", "w", encoding='utf-8') as outfile:
    sys.stdout = outfile
    print(BeautifulSoup(requests.get("https://www.kassoon.com/dnd/5e/premade-characters/").content, "html.parser"))
    sys.stdout = original_stdout

In [ ]:
# pattern testing
pattern = re.search("^([A-Za-z' ]+)", i)

In [139]:
# pattern testing 2
test = ["Darkvision: Thanks to your elf blood, you have superior vision in dark and dim conditions.You can",
       "ends, you gain the following benefits:",
       "hexblade's Curse: Starting at 1st level, you gain the ability to place a baleful curse on someone. As a bonus action, choose",
       "Hex Warrior:At 1st level , you acquire the traini",
       "Thieves’ Cant: You know thieves’ cant, a"]
for t in test:
    pattern = re.search("^([A-Za-z'’ ]+):", t)
    if pattern:
        print(pattern.group(0))
    else:
        print("Pattern Not Found")

Darkvision:
Pattern Not Found
hexblade's Curse:
Hex Warrior:
Thieves’ Cant:


In [ ]:
type(BeautifulSoup)

In [5]:
import webbrowser

chrome_path = "C:\Program Files (x86)\Google\Chrome\Application\chrome.exe %s"

# 1: https://www.kassoon.com/dnd/5e/character-sheet/846/elliza-maren-drow-elf-warlock-1/
# 2: https://www.kassoon.com/dnd/5e/character-sheet/2384/vinheim-high-elf-wizard-1/
# 3: https://www.kassoon.com/dnd/5e/character-sheet/2382/mijira-imbixtellrhyst-dragonborn-rogue-1/
# 4: https://www.kassoon.com/dnd/5e/character-sheet/208/adam-fidge-variant-human-bard-5/

webbrowser.get(chrome_path).open("http://www.kassoon.com/dnd/5e/character-sheet/846/elliza-maren-drow-elf-warlock-1/")
webbrowser.get(chrome_path).open("http://www.kassoon.com/dnd/5e/character-sheet/2384/vinheim-high-elf-wizard-1/")
webbrowser.get(chrome_path).open("http://www.kassoon.com/dnd/5e/character-sheet/2382/mijira-imbixtellrhyst-dragonborn-rogue-1/")
webbrowser.get(chrome_path).open("http://www.kassoon.com/dnd/5e/character-sheet/208/adam-fidge-variant-human-bard-5/")

False